In [1]:
import sys
sys.path.append("..")
import json
from pathlib import Path
import scraper
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
def open_json(path):
    with open(path, "r") as f:
        return json.load(f)

In [4]:
data_dir = Path("../data/T20I/")

# Then for each series we can get the match metadata
for year_path in [path for path in data_dir.iterdir() if path.name=="2015"]:
    with ThreadPoolExecutor() as pool:
        futures=[]
        for series_path in year_path.iterdir():
            with open(series_path / "series_meta.json", "r") as f:
                series_meta = json.load(f)

            with open(series_path / "season_meta.json", "r") as f:
                season_meta = json.load(f)
            
            series_id = series_meta["mappings"]["cricinfo"]
            matches_dir = series_path / "matches"

            if matches_dir.exists():
                for match_path in matches_dir.iterdir():
                    if (match_path / "match_meta.json").exists() and not (match_path / "ball_by_ball.json").exists():
                        try:
                            match_meta = open_json(match_path / "match_meta.json")
                            match_id = match_meta["id"]
                            if match_meta["competitions"][0]["liveAvailable"]:
                                futures.append(pool.submit(scraper.scrape_and_save, scraper.scrape_game, match_path / "ball_by_ball.json", False, series_id, match_id, debug=False))
                                print(match_path)
                                scraper.think_about_it_take_a_second(1)
                            else:
                                print("no comms available for {}".format(match_path))
                            
                        except:
                            print(match_meta)
                            
            scraper.think_about_it_take_a_second(3)

        # wait for everything from year to finish to avoid overloading
        [r.result() for r in as_completed(futures)]
                
                    

../data/testing/T20I/2015/11205-afghanistan-v-oman-t20i-series-2015-16/matches/930583
../data/testing/T20I/2015/11205-afghanistan-v-oman-t20i-series-2015-16/matches/930585
Saved data ../data/testing/T20I/2015/11205-afghanistan-v-oman-t20i-series-2015-16/matches/930583/ball_by_ball.json
Saved data ../data/testing/T20I/2015/11205-afghanistan-v-oman-t20i-series-2015-16/matches/930585/ball_by_ball.json
../data/testing/T20I/2015/11336-pakistan-in-sri-lanka-t20i-series-2015/matches/860281
../data/testing/T20I/2015/11336-pakistan-in-sri-lanka-t20i-series-2015/matches/860279
Saved data ../data/testing/T20I/2015/11336-pakistan-in-sri-lanka-t20i-series-2015/matches/860281/ball_by_ball.json
Saved data ../data/testing/T20I/2015/11336-pakistan-in-sri-lanka-t20i-series-2015/matches/860279/ball_by_ball.json
../data/testing/T20I/2015/11362-nepal-in-netherlands-t20i-series-2015/matches/883347
../data/testing/T20I/2015/11362-nepal-in-netherlands-t20i-series-2015/matches/883341
../data/testing/T20I/2015/